In [5]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for i , pages in enumerate(reader.pages):
        content = pages.extract_text()
        if content:
            text += content + "/n"
        else:
            print(f"page {i+1} has no text")

    return text


In [6]:
book_path = r"/content/drive/MyDrive/Colab Notebooks/6. Inside the Internet of Things (IoT) Author Jonathan Holdowsky.pdf"
book_text = extract_text_from_pdf(book_path)

with open("book_text.txt", "w", encoding="utf-8") as f:
    f.write(book_text)

print("text saved")

page 53 has no text
text saved


In [7]:
# Check number of characters and preview
print("Total extracted length:", len(book_text))
print("\n=== First 1000 characters ===\n")
print(book_text[1000:2000])

# Optional: count how many unique words were extracted
unique_words = len(set(book_text.split()))
print("\nUnique words found:", unique_words)


Total extracted length: 139761

=== First 1000 characters ===

ast seven years, she has been involved in various strategic research assignments for clients in the consumer and industrial products industry.
Michael E. Raynor
Michael E. Raynor is a director with Deloitte Services LP and the director of the Center for Integrated Research (CIR). In collaboration with a broad cross-section of Deloitte professionals from many different industries, the CIR designs, executes, and supports research into some of the most important issues facing companies today.
Mark Cotteleer
Mark Cotteleer  is a research director with Deloitte Services LP , affiliated with Deloitte’s Center for 
Integrated Research. His research focuses on operational and financial performance improvement, in particular, through the application of advanced technology.
AcknowledgementsAbout the authors
Deloitte’s Internet of Things practice enables organizations to identify where the IoT can poten-
tially create value in their i

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def chunk_text(text , chunk_size = 1000 , chunk_overlap = 250):
  splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap,
      separators= ["\n\n" , "\n" , "." , "!" , "?" , " " , ""]
      )
  chunks = splitter.split_text(text)
  return chunks

In [9]:
with open("book_text.txt" , "r" , encoding="utf-8") as f:
  book_text = f.read()
chunks = chunk_text(book_text)
print(len(chunks))

192


In [10]:
import json
with open("book_chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print(chunks[0][:500])

Inside the Internet  
of Things (IoT)
A primer on the technologies building the IoT
/nThe authors would like to thank Sadashiva S.R. (Deloitte Services India Pvt. Ltd.), Dhaval Modi 
(Deloitte Consulting India Pvt. Ltd.), and Joe Mariani (Deloitte Services LP) for their research con-tributions; Gaurav Kamboj  and Kritarth Suri (both with Deloitte Consulting LLP) for their con-
tributions to the IoT technology architectures; and Athappan Balasubramanian (Deloitte Support Services India Pvt. Ltd.)


In [14]:
from InstructorEmbedding import INSTRUCTOR
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model = INSTRUCTOR("hkunlp/instructor-large")

emb_inputs = [["Represent the book text for retrieval:" , chunk] for chunk in chunks]
embddings = model.encode(emb_inputs , show_progress_bar=True)
embddings = np.array(embddings).astype("float32")

print("Generated embeddings with shape:", embddings.shape)



/usr/local/lib/python3.12/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Generated embeddings with shape: (192, 1024)


In [15]:
# Create FAISS index (L2 distance)
dimension = embddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add vectors
index.add(embddings)
print(f"✅ Added {index.ntotal} vectors to FAISS index")

# Optionally save for later
faiss.write_index(index, "book_faiss_index.index")
np.save("book_chunks.npy", np.array(chunks, dtype=object))


✅ Added 192 vectors to FAISS index


In [6]:
import faiss
import numpy as np
import json
from InstructorEmbedding import INSTRUCTOR
from sentence_transformers import SentenceTransformer


# =======================================
# 🔹 Load Model, Index, and Chunks
# =======================================
def load_resources(index_path="/content/book_faiss_index.index", chunks_path="/content/book_chunks.npy"):
    print("📦 Loading FAISS index, chunks, and embedding model...")
    index = faiss.read_index(index_path)
    chunks = np.load(chunks_path, allow_pickle=True)
    model = INSTRUCTOR("hkunlp/instructor-large")
    print(f"✅ Loaded index ({index.ntotal} vectors) and {len(chunks)} chunks.")
    return model, index, chunks

# =======================================
# 🔹 Encode Query
# =======================================
def encode_query(model, query):
    instruction = "Represent the question for retrieving relevant text passages:"
    query_embedding = model.encode([[instruction, query]])
    return query_embedding

# =======================================
# 🔹 Search in FAISS
# =======================================
def search_faiss(index, query_embedding, chunks, k=5):
    distances, indices = index.search(query_embedding, k)
    results = []
    for rank, idx in enumerate(indices[0]):
        results.append({
            "rank": rank + 1,
            "distance": float(distances[0][rank]),
            "text": chunks[idx]
        })
    output_path = "top_chunks.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"✅ Saved top {k} results to '{output_path}'")
    return results

# =======================================
# 🔹 Display Results
# =======================================
def display_results(results):
    print("🔍 Top similar chunks:\n")
    for res in results:
        print(f"Rank {res['rank']} | Distance: {res['distance']:.4f}")
        print(res['text'])
        print("-" * 80)

# =======================================
# 🔹 Example Usage
# =======================================
if __name__ == "__main__":
    model, index, chunks = load_resources()

    query = "What are network protocols."
    query_embedding = encode_query(model, query)
    results = search_faiss(index, query_embedding, chunks, k=5)
    display_results(results)


📦 Loading FAISS index, chunks, and embedding model...


✅ Loaded index (192 vectors) and 192 chunks.
✅ Saved top 5 results to 'top_chunks.json'
🔍 Top similar chunks:

Rank 1 | Distance: 2.7409
ring data from one machine to another via a network is to uniquely identify each of the machines. The IoT requires a unique name for each of the “things” on the network. Network protocols  are a set of rules that define 
how computers identify each other. Broadly, network protocols can be proprietary or open. Proprietary network protocols allow identification and authorization to machines with specific hardware and software, making customization easier and allowing manufac-turers to differentiate their offerings. Open protocols allow interoperability across hetero-geneous devices, thus improving scalability.
26  
Internet Protocol (IP) is an open proto-
--------------------------------------------------------------------------------
Rank 2 | Distance: 2.8001
• Network protocols: Network protocols refer to a set of rules by which machines identify and 

In [8]:
from transformers import AutoTokenizer , AutoModelForCausalLM , pipeline
import json

with open("/content/drive/MyDrive/Colab Notebooks/top_chunks.json" , "r" , encoding="utf-8") as f:
  top_chunks = json.load(f)

context = "\n\n".join([c["text"] for c in top_chunks])

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name , device_map = "auto")

generator = pipeline("text-generation" , model=model , tokenizer = tokenizer)

def generate_answer(question, context):
    prompt = f"""
You are an experienced university professor explaining technical topics clearly and accurately.

Your goal is to answer the question **only using information from the given book context**.
If the answer is not in the context, respond exactly with:
"The book does not discuss this topic."

---
### Context:
{context}

---
### Question:
{question}

---
### Instructions:
- Base your answer strictly on the provided context.
- Explain concepts in a clear, simple, and educational tone.
- Include relevant technical details where appropriate.
- Use an example or analogy only if it helps clarify the concept.
- Keep the explanation concise and focused.
- Do **not** add any external or fabricated information.

---
### Answer:
"""
    result = generator(
        prompt,
        max_new_tokens=300,
        temperature=0.7,
        do_sample=True,
        repetition_penalty=1.2,
        top_k=50,
        top_p=0.9,
    )[0]["generated_text"]

    return result.split("### Answer:")[-1].strip()





Device set to use cuda:0


In [9]:
question = "what are iot protocols"
answer = generate_answer(question, context)

print("🧠 Final Answer:\n", answer)

🧠 Final Answer:
 IoT protocols refers to the set of rules governing how devices communicate with one another within the Internet of Things ecosystem. This includes both traditional networking protocols like TCP/IP and newer ones like MQTT and CoAP. Each IoT protocol has specific capabilities and use cases depending on the type of device involved. Here's a brief overview of some commonly used IoT protocols along with examples:

1. MQTT: A lightweight message broker protocol developed by Mosquitto project. It supports reliable messaging, message encoding, and support for various message formats including JSON, XML, and binary payloads. It’s useful for connecting IoT devices with cloud services and providing real-time data feeds. 2. CoAP: An internet protocol version 7 design aimed specifically at supporting the core function of Internet of Things - distributed coordination of small devices over a shared network. CoAP provides reliable, durable, and transport agnostic APIs for coordinatin